In [ ]:
中文：https://twgreatdaily.com/w4bYyG0BMH2_cNUgurKB.html
英文原文：https://www.dataquest.io/blog/python-excel-xlwings-tutorial/

In [1]:
import xlwings as xw
import pandas as pd

df = pd.read_excel(r'C:\Users\11004076\Documents\Python Scripts\4_Automation\20200324-進銷存-SF4_Vietnam_Silicom.xlsx',
                  sheet_name = 'raw')
df.sample(5)

,進銷存,2nd Item Number,Qty,LargeCat,MediumCat,Sls CD4 Desc#,Product Desc#,Year,Month,Unnamed: 9,進貨量,銷貨量,存貨量
856,存貨量,31731063101,0.0,Speaker,2.0 Speaker,SP-HF160,"RS,SP-HF160,USB,WOOD",2020,2,NaN,NaN,NaN,NaN
439,存貨量,31030114109,354.0,Mouse,Wireless Mouse,NX-7010,"RS,NX-7010,USB,TURQUOISE,BLIS!",2019,9,NaN,NaN,NaN,NaN
326,存貨量,31010116103,700.0,Mouse,Wired Mouse,DX-110,"RS,DX-110,USB,BLUE,G5",2019,3,NaN,NaN,NaN,NaN
615,存貨量,31730007401,0.0,Speaker,BT Speaker,SP-906BT PLUS-LI,"RS,SP-906BT PLUS,R2,Blue,!",2019,11,NaN,NaN,NaN,NaN
808,存貨量,31010116102,724.0,Mouse,Wired Mouse,DX-110,"RS,DX-110,USB,WHITE,G5",2020,2,NaN,NaN,NaN,NaN


In [2]:
wb = xw.Book()
sht = wb.sheets[0]
sht.name = '進銷存'  #改表名
sht.range("A1").value = df  #將df的值貼過來這個新的表(但直接貼會帶有索引)

In [5]:
sht.clear_contents()   #清除此工作表的內容(重做一次)
sht.range("A1").options(index=False).value = df  #並複製不帶索引的數據

### 取得列行

In [ ]:
能夠知道我們的表在哪裡結束是很有用的。更具體地說，我們需要包含了數據的最後一行。為此，我們可以使用end方法和Range對象的row屬性。

不出所料，row方法會返回Range對象的row。

方法end接受一個方向(「up」(或1)、「right」(或2)、「left」(或3)、「down」(或4))作為參數，並返回另一個range對象。
它模仿Excel中常見的CTRL+Shift+箭頭動作。

In [14]:
sht.range("G9").row  #row方法會返回Range對象的row。

9

In [7]:
down_row =sht.range("A1").end('down').row  #最下面那列是excel的第幾列
down_row

1234

In [28]:
df.shape  #Dataframe中的列數

(1233, 13)

In [ ]:
並不是所有的Excel功能都可以作為一個本地xlwings特性來使用。有時我們必須找到變通辦法來做我們想做的事。
幸運的是，xlwings讓這一切變得非常簡單。來自官方文檔中「缺失的特性」一節:

解決方案:
    本質上，xlwings只是一個圍繞Windows平台上的pywin32和Mac平台上的appscript的靈活的包裝器。
    你可以通過調用其api屬性來訪問底層對象。底層對象將使用pywin32語法(感覺很像VBA)和appscript語法(感覺不像VBA)
    為您提供幾乎所有可以用VBA做的事情。但是除了看起來很醜之外，請記住它使您的代碼平台變成特定的(!)。
    Excel Visual Basic for Applications是對各種現有Excel對象的一個詳細解釋。

排序就是xlwings中缺少的功能之一。您可能已經注意到，那些記錄是從最近到最遠的抽籤順序排列的。在接下來的幾個步驟中，我們將反轉順序。

對象ws.range(「A2:N{row}」.format(row=last_row))是一個Range對象。將api屬性追加給它之後，
會生成一個VBA Range對象，該對象反過來又可以訪問它的VBA特性。

我們將使用這個VBA對象的Sort屬性。在它的最簡單的應用程式中，Sort接受兩個參數:
對表排序所使用的列(作為一個VBA Range對象)和排序類型(無論希望按升序還是降序排序)。
第二個參數的參數文檔可以在這裡看到。我們將按升序進行排序。

### 排序

In [120]:
sht.range('A2').options(expand='table').api.Sort(Key1=sht.range("C:C").api, Order2=0,  Header="xlNo")  
#依照C欄排序
#Order1说明：1代表升序，2代表降序

True

In [69]:
sht.range('A2').options(expand='table')

<Range [Book6]進銷存!$A$2>

In [ ]:
sht.range('C2').api.Sort(Key1=sht.range("D2").api, Order1=2,
                         Key2=sht.range("E2").api, Order2=2,
                         Key3=sht.range("C2").api, Order3=2,
                         Header="xlYes") 
#依照C欄排序
#Order1说明：1代表升序，2代表降序

In [121]:
#排序
sht.range('C2').api.Sort(Key1=sht.range("C2").api, Order1=2)

True

In [107]:
sht.range('A2:I1234').api.Sort(Key1=sht.range("C2").api, Order1=2,
                         Key2=sht.range("D2").api, Order2=1,
                         Key3=sht.range("E2").api, Order3=1,
                         Header="xlYes") 

True

In [ ]:
DA

# 公式下拉
在第一個單元格中插入公式後，在常規的Excel工作流中，第二個特性是通過表的末尾自動填充其餘單元格。

### 法一：用迴圈
缺點：1.要到哪一列得自己設定 2.公式不會留著會被貼上值

In [21]:
import xlwings as xw

wb = xw.Book(r'C:\Users\11004076\Documents\Python Scripts\4_Automation\Book1.xlsx')
sheet1 = wb.sheets['Sheet1']

for i in range(3, 100):   
    price_today = sheet1.cells(i, "B").value  # 到 B 欄截取今日收盤價
    price_yesterday = sheet1.cells(i-1, "B").value  # 到 B 欄截取迭代到的上一個 row, 也就是昨日收盤價
    daily_return = (price_today - price_yesterday) / price_yesterday  # 算出該筆資料的日報酬率了   
    sheet1.cells(i, "C").value = daily_return   # 最後再將算出的日報酬率寫入同一列，欄為 C 的儲存格

### 法二：Autofill
在第一個單元格中插入公式後，在常規的Excel工作流中，第二個特性是通過表的末尾自動填充其餘單元格。
Autofill是VBA Range對象的一個方法。它接受兩個參數，即一個將目標單元格作為參數的VBA Range對象和填充類型。
我們感興趣的是枚舉為0的默認值。

In [7]:
import xlwings as xw

wb = xw.Book(r'C:\Users\11004076\Documents\Python Scripts\4_Automation\Book1.xlsx')
sheet2 = wb.sheets['Sheet2']

sheet2.range("C3").value = "=(B3-B2)/B2"

sheet2.range("C3").api.AutoFill(
    sheet2.range("C3:C{row}".format(row=sheet2.range("B1").end('down').row)).api,
    0
)

True

# 實例:Euromillions Data

In [21]:
import pandas as pd
import xlwings as xw
df = pd.read_csv('euromillions.csv')
df.sample(5)

,No.,Day,DD,MMM,YYYY,N1,N2,N3,N4,N5,L1,L2,Jackpot,Wins
942,310,Fri,15,Jan,2010,49,50,26,29,11,4,7,26151124,2
1033,219,Fri,18,Apr,2008,27,6,7,50,3,5,9,46291408,1
1205,47,Fri,31,Dec,2004,8,7,47,25,24,8,9,7074000,0
57,1195,Tue,5,Mar,2019,16,10,46,39,6,8,11,43294160,0
957,295,Fri,2,Oct,2009,44,29,24,23,22,5,1,13764000,0


In [2]:
wb = xw.Book()
ws = wb.sheets["Sheet1"]
ws.name = "EuroMillions"  #改表名
ws.range("A1").options(index=False).value = df  #將df的資料複製到"A1"不帶索引的數據

In [8]:
last_row = ws.range(1,1).end('down').row
print("The last row is {row}.".format(row=last_row))

df.shape  #(1252, 14)
df.shape[0]   #1252 df中的列數(不含標題，所以會比上法少1)
df.shape[1]   #14   df中的欄數
print("The DataFrame df has {rows} rows.".format(rows=df.shape[0]))

The last row is 1253.
The DataFrame df has 1252 rows.


### 排序

In [9]:
#照A欄排序(升序)
ws.range(
"A2:N{row}".format(row=last_row)
).api.Sort(Key1=ws.range("A:A").api, Order1=1)

True

In [10]:
ws.range("O1").value = "Date"
ws.range("O2").value = "=C2&D2&RIGHT(E2, 2)" #寫入excel公式

### 公式向下填滿

In [11]:

ws.range("O2").api.AutoFill(
ws.range("O2:O{row}".format(row=last_row)).api,
0
)

True

In [ ]:
我們還可以使用所需填充類型的命名形式。為此，我們需要從模塊xlwings.constants中檢索它，該模塊包含大多數VBA屬性的枚舉參數的命名版本。
您可以經常通過列印dir(AutoFillType)來檢查可用的屬性。

In [23]:
dir(AutoFillType)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'xlFillCopy',
 'xlFillDays',
 'xlFillDefault',
 'xlFillFormats',
 'xlFillMonths',
 'xlFillSeries',
 'xlFillValues',
 'xlFillWeekdays',
 'xlFillYears',
 'xlGrowthTrend',
 'xlLinearTrend']

In [ ]:
我們在上面所做的也可以通過下面的代碼片段來實現。

In [24]:
#公式向下填滿
from xlwings.constants import AutoFillType
ws.range("O2").api.AutoFill(
ws.range("O2:O{row}".format(row=last_row)).api,
    AutoFillType.xlFillDefault
)

True

### get_address
get_address的Range方法。它接受四個布林參數並返回一個字符串，<br>
該字符串標識不同詳細級別的範圍。

In [65]:
print( ws.range("O2").get_address(0,0) )
print( ws.range("O2").get_address(0,0,0,0) )
print( ws.range("O2").get_address(1,0,0,0) )
print( ws.range("O2").get_address(0,1,0,0) )
print( ws.range("O2").get_address(1,1,0,0) )
print( ws.range("O2").get_address(1,1,1,0) )
print( ws.range("O2").get_address(1,1,1,1) )
print( ws.range("O2").get_address(0,0)[0] )
print( ws.range("O2").get_address(0,0)[1] )
print( ws.range("O12").get_address(0,0)[1:] )

O2
O2
O$2
$O2
$O$2
EuroMillions!$O$2
[Book1]EuroMillions!$O$2
O
2
12


### 自行定義autofill函數
由於我們將經常使用這個操作，我們將創建一個向下填滿函數，給定:<br>
(一個工作表，要被填滿的儲存格，一個需要填充的末尾行)

In [60]:
def autofill(worksheet, cell, last_row):
   rg_cell = worksheet.range(cell)
   to_fill = "{col}{top_row}:{col}{last_row}".format(
      col=rg_cell.get_address(0,0)[0],  #取得欄位的英文(ex:A欄的"A")
      top_row=rg_cell.row,  #取得輸入儲存格(cell)的列數
      last_row=last_row  #自行輸入的數值
   )
   rg_cell.api.Autofill(worksheet.range(to_fill).api, 0)

### 貼上值
為了避免Excel進行不必要的計算，我們將貼上值替換剛才插入到列O中的公式。<br>
由於直接貼上會是水平放置，需要改成垂直放置，使用transpose=True

In [61]:
ws.range('O2').options(transpose=True).value = ws.range('O2:O{row}'.format(row=last_row)).value

In [ ]:
貼上值後，我們可以刪除列C到E，也不會更動公式。

In [62]:
ws.range('C:E').api.Delete()

True

### 刪除不需要的資料

In [ ]:
EuroMillions格式多年來經歷了一些溫和的修改，最後一次修改是在2016年9月24日。

從2016年9月24日起，幸運星的數量從11個數字變為12個數字。為了進行有意義的分析，我們只考慮在最後修改之後進行的抽獎。
下一個代碼片段將查找修改之前的最後一次抽獎，並將其命名為to_delete。

In [92]:
import datetime

for day in ws.range('L2:L{}'.format(last_row)):  #一個一個檢查日期
    if day.value > datetime.datetime(2016, 9, 24, 0, 0): #是否在更改規則之前
        to_delete = int(day.get_address(0, 0)[1:])-1  #我們想要的列是前一列，因此減去1        
        break  # leave the for cycle

In [93]:
to_delete  #No.照排序，940後的日期大於2016/9/24

941

In [ ]:
我們現在可以刪除從第一次抽獎到to_delete的每一行。

In [94]:
ws.range('2:{}'.format(to_delete)).api.Delete()

True

### 格式化

#### 將第一行的字體設置為粗體。

In [95]:
ws.range('1:1').api.Font.Bold = True

#### 自訂數值格式

In [96]:
ws.range('J:J').number_format = "#!.0,萬"

#### 框線

In [97]:
last_column = ws.range(1,1).end('right').get_address(0,0)[0]  #作為後續工作的輔助步驟，我們將找到與最後一列數據對應的字母。 #L

ws.range('A1:{}1'.format(last_column)).api.Borders(9).LineStyle = -4119  #一個雙邊邊框(行樣式-4119，對齊方式9)。

#### 自動調整長寬

In [99]:
ws.autofit()  #現在讓我們對行和列進行自動調整。

#哎呦!這看起來有點壓扁了

In [100]:
#讓我們設置所有列的寬度為J列的寬度，這似乎是最大的值。
ws.range('A:L').api.ColumnWidth = ws.range('J:J').api.ColumnWidth

### 建新工作表

In [ ]:
讓我們add一個名為Frequencies的新空白工作表，並將其分配給Python變量frequencies。

In [101]:
wb.sheets.add('Frequencies')
frequencies = wb.sheets['Frequencies']

In [ ]:
我們將在這個工作表中填入我們剛剛在EuroMillions工作表中處理過的數據集中每個數字和每個幸運星的絕對頻率。

In [102]:
frequencies.range('A1').value = 'Number'   # 加欄名
frequencies.range('A2:A51').value = '=ROW()-1'  #公式:當前列數-1→製造序列

In [ ]:
下面，我們將為單元格B1中的頻率插入一個header，在單元格B2中，我們將輸入一個公式，計算A2中的值在C2:G201範圍內出現的次數。
換句話說，我們將計算1在N1-N5列中出現的次數。在此之後，我們將自動填充B列上的其餘單元格，以對它們各自的行執行相同的操作。

In [103]:
frequencies.range('B1').value = 'Frequency'  # 加欄名
frequencies.range('B2').value = '=COUNTIF(Euromillions!$C$2:$G$201,Frequencies!A2)'
autofill(frequencies, 'B2', 51)  #向下填滿公式:前面的def function

In [ ]:
我們對幸運星也執行同樣的操作:

In [104]:
frequencies.range('D1').value = 'Lucky Star'
frequencies.range('E1').value = 'Frequency'
frequencies.range('D2:D13').value = '=ROW()-1'
frequencies.range('E2').value = '=COUNTIF(EuroMillions!$H$2:$I$201,Frequencies!D2)'
autofill(frequencies, 'E2', 13)
frequencies.autofit()

### 圖表

In [ ]:
創建一個名為Graphs的工作表。

In [105]:
wb.sheets.add('Graphs')
graphs = wb.sheets['Graphs']

In [ ]:
創建一個Chart對象。這只會產生一個空白的白框，我們稍後會用這個方框來圖表化我們的數據。

In [106]:
nr_freq = xw.Chart()

In [107]:
nr_freq.name = 'Number Frequencies'   #我們可以像name工作表一樣來命名圖表
nr_freq.set_source_data(frequencies.range('Frequencies!B1:B51'))  
#方法set_source_data允許我們通過傳入一個範圍對象來定義我們圖表的數據源。
#Excel將嘗試猜測x軸應該是什麼

In [109]:
#設定X軸數列
nr_freq.api[1].SeriesCollection(1).XValues = '=Frequencies!A2:A51'

In [110]:
nr_freq.chart_type = 'column_clustered'  #Excel非常擅長猜測用戶想要什麼樣的圖表，但是為了防止它猜錯，我們將強制它成為柱狀圖。

In [121]:
#現在我們將定義圖表的高度和寬度。度量單位為points。
nr_freq.height = 250
nr_freq.width = 750

In [113]:
nr_freq.api[1].SetElement(2)  # 設置圖表上方的標題
nr_freq.api[1].ChartTitle.Text = 'Number Frequencies'

In [114]:
nr_freq.api[1].HasLegend = 0  #使用HasLegend屬性刪除圖例。

In [ ]:
我們將xlCategory類別作為參數1傳入給Axes方法，並將TickLabelSpacing屬性設置為1，這確保了圖表軸上的每個元素都被顯示。

In [115]:
nr_freq.api[1].Axes(1).TickLabelSpacing = 1

In [ ]:
為了完成對這個圖表的格式化，我們通過

In [116]:
graphs.shapes.api('Number Frequencies').Line.Visible = 0  #將Line對象的Visible屬性設置為0來刪除邊線。

In [ ]:
下面我們對幸運星也做了幾乎同樣的事情。

In [117]:
ls_freq = xw.Chart()
ls_freq.top = 250
ls_freq.name = 'Lucky Star Frequencies'
ls_freq.set_source_data(frequencies.range('Frequencies!E1:E13'))
ls_freq.api[1].SeriesCollection(1).XValues = '=Frequencies!D2:D13' 
ls_freq.chart_type = 'column_clustered'
ls_freq.height = 250
ls_freq.width = 750
ls_freq.api[1].SetElement(2)
ls_freq.api[1].ChartTitle.Text = 'Lucky Star Frequencies'
ls_freq.api[1].HasLegend = 0
ls_freq.api[1].Axes(1).TickLabelSpacing = 1
graphs.shapes.api('Lucky Star Frequencies').Line.Visible = 0

In [ ]:
最後，我們創建了一個顯示jackpot演變的時間序列圖。

In [118]:
jackpot = xw.Chart()
jackpot.top = 500
jackpot.name = 'Jackpot'
last_row = ws.range(1,1).end('down').row
jackpot.set_source_data(ws.range('Euromillions!J2:J{}'.format(last_row)))
#jackpot.api[1].FullSeriesCollection(1).XValues= 'Euromillions!L2:L{}'.format(last_row)   #這串會Error
jackpot.chart_type = 'line'
jackpot.height = 250
jackpot.width = 750
jackpot.api[1].SetElement(2)
jackpot.api[1].ChartTitle.Text = 'Jackpot'
jackpot.api[1].HasLegend = 0
graphs.shapes.api('Jackpot').Line.Visible = 0

In [ ]:
我們通過將TickLabels的NumberForma屬性設置為期望的外觀來修復垂直軸「labels」的格式。

In [119]:
jackpot.api[1].Axes(2).TickLabels.NumberFormat = "#,萬"